<a href="https://colab.research.google.com/github/Carancho25/Challenge-2-Telecom/blob/main/Informe_Final_de_Telecom_2_pynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**I. Manejo de los Datos**

 **I.1 Extracion y visualizacion de los Datos**








In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
datos = pd.read_csv("datos_limpios.csv")


FileNotFoundError: [Errno 2] No such file or directory: 'datos_limpios.csv'

In [ ]:
datos.head(10)


**I.2 Tratamiento de Datos**

**Eliminacion de Columna Customer Id**

In [ ]:
datos = datos.drop('customerID', axis=1)
display(datos.head(10))

In [ ]:
print(datos.isnull().sum())

"Se elimina columna CustomerID, que no aporta para el modelo predictivo y se verifican datos nulos"

In [ ]:
datos.info()

"Identificador de columnas categoricas"

In [ ]:
categorical_cols = datos.select_dtypes(include='object').columns
print("Columnas categóricas identificadas:")
print(categorical_cols)

** Eliminamos la variable -1 en Churn_numeric **

In [ ]:

datos = datos[datos['Churn_numeric'] != -1].copy()

print("\nDimensiones del DataFrame después de eliminar filas con Churn_numeric = -1:", datos.shape)
print("Nueva distribución de 'Churn_numeric' (después de eliminar -1):")
print(datos['Churn_numeric'].value_counts())
print("Porcentaje de cada clase (después de eliminar -1):")
print(datos['Churn_numeric'].value_counts(normalize=True) * 100)

** Convertir account.Charges.Total a numérico y manejar valores nulos **

In [ ]:
datos['account.Charges.Total'] = pd.to_numeric(datos['account.Charges.Total'], errors='coerce')

In [ ]:
datos['account.Charges.Total'].fillna(datos['account.Charges.Total'].mean(), inplace=True)

print("\nTipo de dato de 'account.Charges.Total' DESPUÉS de la conversión y manejo de NaN:", datos['account.Charges.Total'].dtype)
print("Descripción estadística de 'account.Charges.Total' después de la conversión:")
print(datos['account.Charges.Total'].describe())
print("\nInformación general del DataFrame (para ver el tipo de dato de todas las columnas):")
datos.info()

** Verificacion del porcentaje de cancelacion (Churn) **

** Mostrar la distribución de la variable 'Churn_numeric, para verificar si hay desbalance **

In [ ]:
print("Distribución de la variable 'Churn_numeric':")
print(datos['Churn_numeric'].value_counts())
print("\nPorcentaje de cada clase:")
print(datos['Churn_numeric'].value_counts(normalize=True) * 100)

** Eliminacion de la Columna Churn, dado que ya se tiene Churn_numeric para evitar informacion confusa **


In [ ]:
if 'Churn' in datos.columns:
    datos.drop('Churn', axis=1, inplace=True)
    print("Columna 'Churn' original (texto) eliminada.")

** Aplicar One-Hot Encoding a las columnas categóricas restantes
**

In [ ]:
categorical_cols = datos.select_dtypes(include='object').columns

print("\nColumnas categóricas identificadas para One-Hot Encoding:", categorical_cols.tolist())

# Aplicamos One-Hot Encoding.
# 'drop_first=True' evita la multicolinealidad, eliminando una de las columnas binarias
# creadas para cada variable original. Por ejemplo, para 'Gender', si tienes 'Female' y 'Male',
# solo se crearía 'Gender_Male' (si es 1, es Male; si es 0, es Female).
df_encoded = pd.get_dummies(datos, columns=categorical_cols, drop_first=True)

## II. Analisis de Correlacion: Matriz de Correlacion ##


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

correlation_matrix = df_encoded.corr()

# Plot the heatmap
plt.figure(figsize=(18, 14))
sns.heatmap(correlation_matrix, cmap='coolwarm', annot=False)
plt.title('Correlation Matrix')
plt.show()

** Relacion de la variable independiente con el resto de variables **

In [ ]:

# calculo de la matriz de correlacion #
correlation_matrix = df_encoded.corr()

churn_correlations = correlation_matrix['Churn_numeric'].sort_values(key=abs, ascending=False)
churn_correlations = churn_correlations.drop('Churn_numeric')
print("Mayores correlaciones con Churn_numeric (Ordenado por absolute value):")
print(churn_correlations)

In [ ]:
# Mostrar porcentajes en orden ascendente y como porcentajes #
churn_correlations_percentage_asc = (churn_correlations * 100).sort_values(ascending=True)

print("Correlaciones con Churn_Yes (en porcentajes, ascending order):")
print(churn_correlations_percentage_asc)

** Analisis dirigido a variables especificas : Duracion del contrato y gasto total con la variable Churn_numeric **

In [ ]:

if 'correlation_matrix' in locals():
    # correlacion entre 'customer.tenure' y 'Churn_numeric'
    tenure_churn_correlation = correlation_matrix.loc['customer.tenure', 'Churn_numeric']
    print(f"La correlación entre 'customer.tenure' y 'Churn_numeric' es: {tenure_churn_correlation:.4f}")


# **Visualizacion de los patrones  en el dataset usando graficos boxplots y scatter plots.**##

In [ ]:

numerical_cols = df_encoded.select_dtypes(include=['float64', 'int64']).columns.tolist()


categorical_cols_for_plotting = datos.select_dtypes(include='object').columns.tolist()


if 'Churn_numeric' in numerical_cols:
    numerical_cols.remove('Churn_numeric')


if 'Churn' in categorical_cols_for_plotting:
    categorical_cols_for_plotting.remove('Churn')
if 'Churn_numeric' in categorical_cols_for_plotting:
    categorical_cols_for_plotting.remove('Churn_numeric')


print("Columnas numericas identificadas para plotting:")
print(numerical_cols)
print("\nColumnas categoricas identificadas para plotting:")
print(categorical_cols_for_plotting)


In [ ]:
# Selecion de categorias  para plotting basados en el analisís de correlacion
selected_categorical_cols = [
    'internet.InternetService',
    'account.Contract',
    'account.PaymentMethod',
    'customer.Partner',
    'customer.Dependents',
    'customer.SeniorCitizen',
    'account.PaperlessBilling'
]

# Definimos la  variable objetivo
target_variable = 'Churn_numeric'

# Creamos los  boxplots correspondientes
for col in selected_categorical_cols: # Add the loop here
    plt.figure(figsize=(8, 6))
    sns.boxplot(x=col, y=target_variable, data=datos)
    plt.title(f'Distribución de Churn por {col}')
    plt.xlabel(col)
    plt.ylabel('Churn (0: No, 1: Yes)')
    plt.show()

## Generacion de scatter  plots ##





In [ ]:
# Selecion de pares de caracteristicas numericas para crear scatter plots

numerical_cols = df_encoded.select_dtypes(include=['float64', 'int64']).columns.tolist()
# Eliminacion de la variable R Churn_numeric de la lista, dado que es "target"
if 'Churn_numeric' in numerical_cols:
    numerical_cols.remove('Churn_numeric')

# Se define los pares mas relevantes en base al analisis de correlacion y a las variables selecionadas:
selected_pairs = [
    ('customer.tenure', 'account.Charges.Monthly'),
    ('customer.tenure', 'account.Charges.Total'),
    ('account.Charges.Monthly', 'account.Charges.Total'),
    ('customer.tenure', 'Cuentas_Diarias'),
    ('account.Charges.Monthly', 'Cuentas_Diarias'),
    ('account.Charges.Total', 'Cuentas_Diarias'),
    ('customer.SeniorCitizen', 'customer.tenure')
]

# Creacion de los  scatter plots para las variables selecionadas:
for col1, col2 in selected_pairs:
    plt.figure(figsize=(10, 6))
    sns.scatterplot(x=col1, y=col2, data=df_encoded)
    plt.title(f'Scatter Plot of {col1} vs {col2}')
    plt.xlabel(col1)
    plt.ylabel(col2)
    plt.grid(True)
    plt.show()

## Interpretacion de los graficos, para identificar patrones potenciales y relaciones de la informacion ##



## Resumen:

### Hallazgos Clave del Análisis de Datos

* Los clientes con servicio de internet de fibra óptica, contratos mes a mes y métodos de pago de cheque electrónico parecen tener una tasa de abandono mediana más alta según los diagramas de caja.

* Los diagramas de dispersión muestran relaciones distintas entre las características numéricas, como una correlación positiva entre `customer.tenure` y `account.Charges.Total`, y entre `account.Charges.Monthly` y `account.Charges.Total`.

* El diagrama de dispersión de `customer.SeniorCitizen` frente a `customer.tenure` sugiere que los clientes mayores pueden tener distribuciones de permanencia ligeramente diferentes en comparación con los no mayores.



# *III Modelado Predictivo* #

# Separacion de Datos #

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X = df_encoded.drop('Churn_numeric', axis=1)
y = df_encoded['Churn_numeric']
print(f"\nDimensiones de X (Características): {X.shape}")
print(f"Dimensiones de y (Variable Objetivo): {y.shape}")

# Dividir datos de entrenamiento y prueba #

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# III.Creacion de Modelos: #


# Aplicacion de Smote al conjunto de entrenamiento"

In [ ]:
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE # Necesitas haber instalado 'imbalanced-learn'
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier


In [ ]:
sm = SMOTE(random_state=42) # random_state para reproducibilidad
X_train_res, y_train_res = sm.fit_resample(X_train, y_train)

print("\nDimensiones de X_train_res después de SMOTE:", X_train_res.shape)
print("Dimensiones de y_train_res después de SMOTE:", y_train_res.shape)
print("\nDistribución de 'Churn_numeric' en y_train_res DESPUÉS de SMOTE:")
print(y_train_res.value_counts())
print(y_train_res.value_counts(normalize=True) * 100)

# III.1 Aplicacion del primer modelo de Regresion Logistica #

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

log_reg_model = LogisticRegression(random_state=42, solver='liblinear', max_iter=1000)

print("Entrenando el modelo de Regresión Logística...")
log_reg_model.fit(X_train_res, y_train_res)
print("Modelo de Regresión Logística entrenado con éxito!\n")

Realizar Predicciones en el conjunto de prueba (X_test)

In [ ]:
y_pred_log_reg = log_reg_model.predict(X_test) # Predicciones de clase (0 o 1)
y_proba_log_reg = log_reg_model.predict_proba(X_test)[:, 1] # Probabilidad de la clase '1' (Churn)


Evaluar el Modelo

In [ ]:
# Matriz de Confusión: Nos muestra los aciertos y errores detalladamente.
cm_log_reg = confusion_matrix(y_test, y_pred_log_reg)
print("\nMatriz de Confusión:")
print(cm_log_reg)

# Visualización de la Matriz de Confusión
plt.figure(figsize=(6, 5))
sns.heatmap(cm_log_reg, annot=True, fmt='d', cmap='Blues',
            xticklabels=['No Churn (Pred)', 'Churn (Pred)'],
            yticklabels=['No Churn (Real)', 'Churn (Real)'])
plt.title('Matriz de Confusión - Regresión Logística')
plt.xlabel('Predicción')
plt.ylabel('Valor Real')
plt.show()

# Reporte de Clasificación: Proporciona Precision, Recall y F1-score por clase.
print("\nReporte de Clasificación:")
print(classification_report(y_test, y_pred_log_reg, target_names=['No Churn', 'Churn']))

# AUC-ROC Score: Mide la capacidad del modelo para distinguir entre las clases.
auc_roc_log_reg = roc_auc_score(y_test, y_proba_log_reg)
print(f"AUC-ROC Score: {auc_roc_log_reg:.4f}")

# Interpretacion de Resultados del Modelo de Regresion Lineal #
[0,0] (arriba izquierda): Verdaderos Negativos (TN) - Clientes que no hicieron churn y el modelo predijo correctamente que no lo harían.

[0,1] (arriba derecha): Falsos Positivos (FP) - Clientes que no hicieron churn, pero el modelo predijo erróneamente que sí lo harían.

[1,0] (abajo izquierda): Falsos Negativos (FN) - Clientes que sí hicieron churn, pero el modelo predijo erróneamente que no lo harían.

[1,1] (abajo derecha): Verdaderos Positivos (TP) - Clientes que sí hicieron churn y el modelo predijo correctamente que sí lo harían.


 # Informe de Clasificacion #


** Clase 'No Churn' (0):

Precisión (0.87): De todas las predicciones de "No Churn", el 87% fueron correctas.

Recall (0.81): El modelo identificó correctamente al 81% de todos los clientes que realmente no hicieron Churn.

Clase 'Churn' (1):

Precisión (0.56): De todas las predicciones de "Churn", solo el 56% fueron correctas. Esto significa que hay un número considerable de "falsas alarmas".

Recall (0.65): De todos los clientes que realmente hicieron Churn, el modelo logró identificar al 65%. Esta es una métrica clave para un problema de Churn, ya que SE quiere detectar a la mayor cantidad posible de clientes en riesgo (Churn).

F1-Score (0.60): Un balance entre precisión y recall para la clase Churn.

AUC-ROC Score :

AUC-ROC Score: 0.8228

Este valor es bueno. Indica que el modelo tiene una buena capacidad general para distinguir entre los clientes que harán Churn y los que no.

** Conclusión del Modelo de Regresión Logística:**
El modelo de Regresión Logística, entrenado con los datos balanceados por SMOTE, se muestra con un rendimiento razonable:

Es capaz de detectar a una proporción decente de clientes que realmente abandonarán (Recall del 65% para 'Churn').

Tiene un buen poder discriminatorio general (AUC-ROC de 0.82).

Analisis Critico: Un tema relevante para asegurar la confiabilidad del modelo es poder mejorar es la precisión para la clase 'Churn'. Un 56% de precisión significa que cerca de  la mitad de las veces que el modelo predice "Churn", en realidad se equivoca. Esto podría llevar a acciones de retención dirigidas a clientes que no estaban en riesgo real.

## III.2 Aplicacion de un segundo Modelo : Random Forest Classifier ##

In [ ]:
!pip install imbalanced-learn

In [ ]:
import pandas as pd
import io
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier # Importamos el modelo Random Forest
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
X = df_encoded.drop('Churn_numeric', axis=1)
y = df_encoded['Churn_numeric']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

sm = SMOTE(random_state=42)
X_train_res, y_train_res = sm.fit_resample(X_train, y_train)

print("Datos de entrenamiento balanceados con SMOTE listos para el modelado de Random Forest.")
print(f"Dimensiones de X_train_res: {X_train_res.shape}")
print(f"Dimensiones de y_train_res: {y_train_res.shape}\n")

In [ ]:
from sklearn.ensemble import RandomForestClassifier # Importamos el modelo Random Forest
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
rf_model = RandomForestClassifier(n_estimators=100, random_state=42) # 100 árboles por defecto

print("Entrenando el modelo Random Forest con los datos balanceados...")
rf_model.fit(X_train_res, y_train_res)
print("¡Modelo Random Forest entrenado con éxito!\n")

In [ ]:
# --- 2. Realizar Predicciones en el conjunto de prueba ---
# Usamos el conjunto de prueba ORIGINAL para una evaluación justa.
y_pred_rf = rf_model.predict(X_test) # Predicciones de clase (0 o 1)
y_proba_rf = rf_model.predict_proba(X_test)[:, 1] # Probabilidad de la clase '1' (Churn)

In [ ]:
# Matriz de Confusión
cm_rf = confusion_matrix(y_test, y_pred_rf)
print("\nMatriz de Confusión:")
print(cm_rf)

In [ ]:
# Visualización de la Matriz de Confusión
plt.figure(figsize=(6, 5))
sns.heatmap(cm_rf, annot=True, fmt='d', cmap='Blues',
            xticklabels=['No Churn (Pred)', 'Churn (Pred)'],
            yticklabels=['No Churn (Real)', 'Churn (Real)'])
plt.title('Matriz de Confusión - Random Forest')
plt.xlabel('Predicción')
plt.ylabel('Valor Real')
plt.show()

In [ ]:
# Reporte de Clasificación
print("\nReporte de Clasificación:")
print(classification_report(y_test, y_pred_rf, target_names=['No Churn', 'Churn']))

# AUC-ROC Score
auc_roc_rf = roc_auc_score(y_test, y_proba_rf)
print(f"AUC-ROC Score: {auc_roc_rf:.4f}")

# Aplicacion de un tercer modelo :  Arbol de Decision ##


In [ ]:
import pandas as pd
import io
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.tree import DecisionTreeClassifier # Importamos el modelo Árbol de Decisión
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:

X = df_encoded.drop('Churn_numeric', axis=1)
y = df_encoded['Churn_numeric']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

sm = SMOTE(random_state=42)
X_train_res, y_train_res = sm.fit_resample(X_train, y_train)

print("Datos de entrenamiento balanceados con SMOTE listos para el modelado de Árbol de Decisión.")
print(f"Dimensiones de X_train_res: {X_train_res.shape}")
print(f"Dimensiones de y_train_res: {y_train_res.shape}\n")

In [ ]:
from sklearn.tree import DecisionTreeClassifier # Importamos el modelo Árbol de Decisión
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
import matplotlib.pyplot as plt
import seaborn as sns


dt_model = DecisionTreeClassifier(random_state=42, max_depth=7) # Profundidad es 10

print("Entrenando el modelo Árbol de Decisión con los datos balanceados...")
dt_model.fit(X_train_res, y_train_res)
print("¡Modelo Árbol de Decisión entrenado con éxito!\n")


# --- 2. Realizar Predicciones en el conjunto de prueba ---
# Usamos el conjunto de prueba ORIGINAL para una evaluación justa.
y_pred_dt = dt_model.predict(X_test) # Predicciones de clase (0 o 1)
y_proba_dt = dt_model.predict_proba(X_test)[:, 1] # Probabilidad de la clase '1' (Churn)


# --- 3. Evaluar el Modelo ---
print("--- Evaluación del Modelo Árbol de Decisión ---")

# Matriz de Confusión
cm_dt = confusion_matrix(y_test, y_pred_dt)
print("\nMatriz de Confusión:")
print(cm_dt)

# Visualización de la Matriz de Confusión
plt.figure(figsize=(6, 5))
sns.heatmap(cm_dt, annot=True, fmt='d', cmap='Blues',
            xticklabels=['No Churn (Pred)', 'Churn (Pred)'],
            yticklabels=['No Churn (Real)', 'Churn (Real)'])
plt.title('Matriz de Confusión - Árbol de Decisión')
plt.xlabel('Predicción')
plt.ylabel('Valor Real')
plt.show()

# Reporte de Clasificación
print("\nReporte de Clasificación:")
print(classification_report(y_test, y_pred_dt, target_names=['No Churn', 'Churn']))

# AUC-ROC Score
auc_roc_dt = roc_auc_score(y_test, y_proba_dt)
print(f"AUC-ROC Score: {auc_roc_dt:.4f}")

Interpretacion del Modelo de Árbol de Decisión (con 31 Características):
Matriz de Confusión Implícita (a partir del Reporte):

a. Verdaderos Positivos (TP): Clientes de Churn predichos correctamente. (Calculado: Recall * Support para Churn = 0.68 * 561 = ~381)

b.Falsos Negativos (FN): Clientes de Churn no detectados. (Calculado: Support - TP para Churn = 561 - 381 = ~180)

c.Falsos Positivos (FP): Clientes de No Churn predichos como Churn (falsas alarmas). (Calculado: TP * (1/Precision - 1) para Churn = 381 * (1/0.52 - 1) = ~352)

d.Verdaderos Negativos (TN): Clientes de No Churn predichos correctamente. (Calculado: Support - FP para No Churn = 1552 - 352 = ~1200)

**Métricas Clave (Clase 'Churn' - lo que es más relevante en el modelo):**

a.Precision (Churn): 0.52

Esto significa que de todas las veces que el modelo predijo que un cliente haría Churn, fue correcto solo el 52% de las veces. El 48% restante fueron "falsas alarmas" (predijo Churn, pero el cliente no lo hizo). Esto puede ser costoso si las acciones de retención son caras, de cara a lo que buca la empresa.

b.Recall (Churn): 0.68

De todos los clientes que realmente hicieron Churn (561 en el conjunto de prueba), el modelo logró identificar correctamente al 68%. Este es un buen Recall, indicando que el modelo es bastante efectivo para "capturar" a los clientes en riesgo real.

c.F1-Score (Churn): 0.59

Es la media armónica de precisión y recall. Un F1-Score de 0.59 estaria indicando un rendimiento moderado para la clase minoritaria.

Métricas Generales:

**Accuracy (Precisión Global): 0.75**

El modelo clasificó correctamente al 75% de todos los clientes en el conjunto de prueba.

**AUC-ROC Score: 0.8054**

Este valor de 0.8054 se considera aceptable. Sugiere que el modelo tiene una buena capacidad general para distinguir entre las clases de Churn y No-Churn, aunque no es tan alta como la del Random Forest que se calculo anteriormente.

In [ ]:
print("--- Modelo: Regresión Logística ---")
log_reg_model = LogisticRegression(random_state=42, solver='liblinear', max_iter=1000)
log_reg_model.fit(X_train_res, y_train_res)
y_pred_log_reg = log_reg_model.predict(X_test)
y_proba_log_reg = log_reg_model.predict_proba(X_test)[:, 1]

print("\nMatriz de Confusión (Regresión Logística):")
print(confusion_matrix(y_test, y_pred_log_reg))
print("\nReporte de Clasificación (Regresión Logística):")
print(classification_report(y_test, y_pred_log_reg, target_names=['No Churn', 'Churn']))
print(f"AUC-ROC Score (Regresión Logística): {roc_auc_score(y_test, y_proba_log_reg):.4f}\n")


# --- 2. Random Forest Classifier ---
print("--- Modelo: Random Forest Classifier ---")
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train_res, y_train_res)
y_pred_rf = rf_model.predict(X_test)
y_proba_rf = rf_model.predict_proba(X_test)[:, 1]

print("\nMatriz de Confusión (Random Forest):")
print(confusion_matrix(y_test, y_pred_rf))
print("\nReporte de Clasificación (Random Forest):")
print(classification_report(y_test, y_pred_rf, target_names=['No Churn', 'Churn']))
print(f"AUC-ROC Score (Random Forest): {roc_auc_score(y_test, y_proba_rf):.4f}\n")


# --- 3. Árbol de Decisión ---
print("--- Modelo: Árbol de Decisión ---")
dt_model = DecisionTreeClassifier(random_state=42, max_depth=7)
dt_model.fit(X_train_res, y_train_res)
y_pred_dt = dt_model.predict(X_test)
y_proba_dt = dt_model.predict_proba(X_test)[:, 1]

print("\nMatriz de Confusión (Árbol de Decisión):")
print(confusion_matrix(y_test, y_pred_dt))
print("\nReporte de Clasificación (Árbol de Decisión):")
print(classification_report(y_test, y_pred_dt, target_names=['No Churn', 'Churn']))
print(f"AUC-ROC Score (Árbol de Decisión): {roc_auc_score(y_test, y_proba_dt):.4f}\n")

print("--- Evaluación Comparativa de modelos ---")

** Tabla Comparativa de Modelos **

In [ ]:
# Resultados del modelo de Regresión Logística
log_reg_metrics = {
    'Precision': 0.56,
    'Recall': 0.65,
    'F1-Score': 0.60,
    'AUC-ROC Score': 0.8228,
    'Accuracy Global': 0.77
}

# Resultados del modelo de Random Forest
rf_metrics = {
    'Precision': 0.57,
    'Recall': 0.59,
    'F1-Score': 0.58,
    'AUC-ROC Score': 0.8202,
    'Accuracy Global': 0.77
}

# Resultados del modelo de Árbol de Decisión
dt_metrics = {
    'Precision': 0.52,
    'Recall': 0.68,
    'F1-Score': 0.59,
    'AUC-ROC Score': 0.8054,
    'Accuracy Global': 0.75
}

# Crea un DataFrame a partir de los diccionarios
comparison_df = pd.DataFrame({
    'Regresión Logística': log_reg_metrics,
    'Random Forest Classifier': rf_metrics,
    'Árbol de Decisión': dt_metrics
})

# Muestra el cuadro comparativo
print("Cuadro Comparativo de Modelos de Clasificación:")
print(comparison_df)

**Tabla Comparativa de Rendimiento de Modelos**
Métrica / Modelo
Regresión Logística (RL) /	Random Forest (RF) Classifier	/Árbol de Decisión (AD)
                     RL      RF      AD
a.Churn Precision:	0.56	  0.57	  0.52
b.Churn Recall:   	0.65	  0.59	  0.68
c.Churn F1-Score :	0.60	  0.58	  0.59
d.AUC-ROC Score:  	0.8228	0.8202	0.8054
e.Accuracy Global:	0.77	  0.77	  0.75
Support (No Churn)	1552	1552	1552
Support (Churn)	561	561	561

Análisis y Comparación Final de los Modelos
**1. Regresión Logística**
Pros: Es simple, rápido, interpretable.
Contras: A veces limitado por su naturaleza lineal.


1.1. Churn Precision (0.56): Es la mejor entre los tres para este set de características, lo que significa menos "falsas alarmas" al identificar evasion.

1.2 Churn Recall (0.65): Es muy bueno, capturando una gran parte de los evasores reales.

1.3 Churn F1-Score (0.60): El más alto, indicando el mejor equilibrio entre precisión y recall para la clase 'Churn'.

1.4 AUC-ROC (0.8228): El más alto de los tres, mostrando la mejor capacidad discriminatoria general.

1.5 Accuracy (0.77): Comparte la precisión global más alta con Random Forest.

**2. Random Forest Classifier**
Pros: Potente, robusto, maneja bien datos complejos.
Contras: Menos interpretable, puede ser más lento.


2.1 Churn Precision (0.57): Ligeramente mejor que Regresión Logística, pero la diferencia es mínima.

2.2. Churn Recall (0.59): El más bajo de los tres, lo que significa que "pierde" a más clientes que realmente hacen churn.

2.3 Churn F1-Score (0.58): El más bajo, indicando un equilibrio menos favorable para la clase 'Churn'.

2.4 AUC-ROC (0.8202): Muy cercano a Regresión Logística.

2.5 Accuracy (0.77): Comparte la precisión global más alta.

**3. Árbol de Decisión**
Pros: Muy interpretable, fácil de visualizar.

Contras: Muy propenso al sobreajuste si no se le controla bien (max_depth). Generalmente, un solo árbol no es tan robusto como un "bosque".


3.1. Churn Precision (0.52): La más baja de los tres.

3.2.Churn Recall (0.68): El más alto de los tres, lo que significa que es el mejor para capturar a la mayoría de los churners reales, aunque a costa de muchas falsas alarmas.

3.2. Churn F1-Score (0.59): Intermedio.

3.3. AUC-ROC (0.8054): El más bajo de los tres.

3.4. Accuracy (0.75): La más baja de los tres.

¿Cuál sería el mejor modelo para la variable 'Churn' (con 31 características)?
Dado el objetivo de predecir el Churn, donde generalmente se busca un balance entre identificar a los clientes que se van (Recall) y hacerlo de manera eficiente sin demasiadas falsas alarmas (Precision), y considerando el poder discriminatorio general (AUC-ROC):

El modelo de Regresión Logística se destaca como el mejor modelo en este conjunto específico de 31 características.

Justificación:

Mejor F1-Score para Churn (0.60): Indica que ofrece el mejor equilibrio entre la precisión y el recall para la clase 'Churn'.

Mejor AUC-ROC Score (0.8228): Muestra que tiene la mejor capacidad general para distinguir entre los clientes que harán churn y los que no, lo cual es fundamental para el problema.

Buena Precisión (0.56) y Recall (0.65) para Churn: Aunque su precisión no es extremadamente alta, es la mejor de los tres, y su recall es muy bueno. Esto implica que podrás identificar una buena parte de los churners reales con una tasa de falsas alarmas más manejable que con el Árbol de Decisión.

Aunque el Random Forest también tiene un buen rendimiento general, su Recall para la clase Churn fue el más bajo en esta comparación (0.59), lo que significa que "perdería" a más clientes que realmente van a hacer churn. El Árbol de Decisión, aunque tiene un alto recall, su baja precisión lo hace menos eficiente en un escenario real donde las acciones de retención tienen un costo.



## Conclusiones Finales y Recomendaciones en base al Modelo Selecionado : Regresion Lineal ##

** Variables Claves del Modelo a considerar **

# Variables Claves del Modelo #

*   Elemento de lista

Según orden de importancia y lo que representan para Telecom:

1.) account.Contract_Month-to-month (Contrato Mes a Mes):
Es la variable más fuerte de Churn (evasion).Los clientes con contratos mensuales tienen una probabilidad significativamente más alta de abandonar la empresa.La interpretacion es que la falta de compromiso a largo plazo los hace más volátiles y propensos a cambiar.

2.) internet.InternetService_Fiber optic (Servicio de Internet de Fibra Óptica):
Esta variable tambien aumenta la probabilidad de Churn .A pesar de ser un servicio premium, los clientes con fibra óptica son más propensos a irse. Esto podría estar indicando problemas de calidad, estabilidad, soporte técnico insatisfactorio o expectativas no cumplidas para este servicio de alta velocidad.

3.) account.PaymentMethod_Electronic check . Esta variable tambien incide en aumentar la probabilidad de Churn. De esta manera, los clientes que pagan con cheque electrónico tienen un mayor riesgo de abandono. Esto podría estar implicando en ctegorizar a señalar a un segmento de clientes menos bancarizados, o con procesos de pago que generan fricción o insatisfacción en general.

4.) customer.tenure (Antigüedad del Cliente):
Esta variable al presentar una correlacion negativa en el modelo, hace que disminuye la probabilidad de Churn . De esta manera, a mayor cantidad de tiempo que el cliente se mantiena en la empresa, menos probable es que se vaya.Esto implica en la practicaLa que los clientes más nuevos son más vulnerables.

## Conclusiones Estrategicas y Recomendaciones Finales ##

I.) Incentivar Contratos a Largo Plazo:

Estrategia Sugerida: Diseñar programas agresivos para migrar a clientes de contrato "Mes a Mes" a contratos de uno o dos años,mediante ofertas de fidelización, tales como: Descuentos atractivos, meses gratis, etc.


II.) Mejorar la Experiencia del Cliente de Fibra Óptica:

Estrategia Sugerida: Investigar y resolver los puntos de dolor específicos de los usuarios de fibra óptica, mediante el monitoreo proactivo, es decir implementar sistemas para detectar automáticamente problemas de rendimiento o interrupciones en el servicio de fibra.


III:) Optimizar y Desincentivar el Pago con Cheque Electrónico:

Estrategia Sugerida: Reducir la dependencia del cheque electrónico y facilitar métodos de pago más convenientes y estables, mediante incentivos tales como el ofrecer pequeños descuentos o bonos por migrar a pagos automáticos con tarjeta de crédito/débito o transferencia bancaria.


IV:) Programas de Bienvenida y Fidelización Temprana (Antigüedad):

Estrategia Sugerida: Reconocer que los primeros meses son críticos para la retención y por lo tanto ofrecer
un proceso de bienvenida que asegure que el cliente entienda y utilice todos los servicios. Tambien se sugiere ofrecer ofertas post-bienvenida: Pequeños incentivos o encuestas de satisfacción después de 3 o 6 meses.